In [1]:
!pip install dash plotly

  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 27.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 7.7 MB/s eta 0:00:00
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import numpy as np

# Sample Data
np.random.seed(0)
type1_data = np.random.randint(1, 4, size=100)
type2_data = np.random.randint(2, 5, size=100)
type3_data = np.random.randint(1, 6, size=100)

# App Setup
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Interactive Histogram with Filters"),
    
    # Filter Checkbox
    html.Div([
        html.Label("Select Data Types:"),
        dcc.Checklist(
            id='data-type-filter',
            options=[
                {'label': 'Type 1', 'value': 'type1'},
                {'label': 'Type 2', 'value': 'type2'},
                {'label': 'Type 3', 'value': 'type3'}
            ],
            value=['type1'],  # Default selected
            labelStyle={'display': 'inline-block'}
        )
    ]),
    
    # Graph
    dcc.Graph(id='histogram')
])

# Callback to update the graph based on selected filters
@app.callback(
    Output('histogram', 'figure'),
    [Input('data-type-filter', 'value')]
)
def update_histogram(selected_types):
    data = []

    # Add data traces based on the selected types
    if 'type1' in selected_types:
        data.append(go.Histogram(x=type1_data, name='Type 1', opacity=0.75))
    if 'type2' in selected_types:
        data.append(go.Histogram(x=type2_data, name='Type 2', opacity=0.75))
    if 'type3' in selected_types:
        data.append(go.Histogram(x=type3_data, name='Type 3', opacity=0.75))

    layout = go.Layout(
        title='Histogram of Different Types',
        xaxis=dict(title='Values'),
        yaxis=dict(title='How Many People'),
        barmode='overlay'
    )

    return {'data': data, 'layout': layout}

# Run the App
if __name__ == '__main__':
    app.run_server(debug=True)


In [4]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

# Sample data
before_data = {
    'type1': [1, 2, 3, 4, 5],
    'type2': [2, 3, 4, 5, 6],
    'type3': [3, 4, 5, 6, 7]
}

after_data = {
    'type1': [2, 3, 4, 5, 6],
    'type2': [3, 4, 5, 6, 7],
    'type3': [4, 5, 6, 7, 8]
}

# Calculate difference data
difference_data = {
    'type1': [after - before for before, after in zip(before_data['type1'], after_data['type1'])],
    'type2': [after - before for before, after in zip(before_data['type2'], after_data['type2'])],
    'type3': [after - before for before, after in zip(before_data['type3'], after_data['type3'])]
}

# Calculate total data
def calculate_total(data_dict):
    return [sum(x) for x in zip(data_dict['type1'], data_dict['type2'], data_dict['type3'])]

total_data = {
    'before': calculate_total(before_data),
    'after': calculate_total(after_data),
    'difference': calculate_total(difference_data)
}

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Interactive Histogram with Filters and Time Selection"),
    
    # Dropdown for selecting data type
    dcc.Dropdown(
        id='data-type-filter',
        options=[
            {'label': 'Type 1', 'value': 'type1'},
            {'label': 'Type 2', 'value': 'type2'},
            {'label': 'Type 3', 'value': 'type3'},
            {'label': 'Total', 'value': 'total'}
        ],
        value=['type1', 'type2', 'type3', 'total'],
        multi=True
    ),
    
    # Dropdown for selecting "Before", "After", "Difference"
    dcc.Dropdown(
        id='time-filter',
        options=[
            {'label': 'Before', 'value': 'before'},
            {'label': 'After', 'value': 'after'},
            {'label': 'Difference', 'value': 'difference'}
        ],
        value='before'
    ),
    
    # Graph
    dcc.Graph(id='histogram')
])

# Callback to update the graph based on selected filters
@app.callback(
    Output('histogram', 'figure'),
    [Input('data-type-filter', 'value'),
     Input('time-filter', 'value')]
)
def update_histogram(selected_types, time_filter):
    data = []

    # Select the correct data based on the time filter
    if time_filter == 'before':
        current_data = before_data
        current_total = total_data['before']
    elif time_filter == 'after':
        current_data = after_data
        current_total = total_data['after']
    elif time_filter == 'difference':
        current_data = difference_data
        current_total = total_data['difference']
    else:
        current_data = before_data
        current_total = total_data['before']

    # Add data traces based on the selected types
    if 'type1' in selected_types:
        data.append(go.Histogram(x=current_data['type1'], name='Type 1', opacity=0.75))
    if 'type2' in selected_types:
        data.append(go.Histogram(x=current_data['type2'], name='Type 2', opacity=0.75))
    if 'type3' in selected_types:
        data.append(go.Histogram(x=current_data['type3'], name='Type 3', opacity=0.75))
    if 'total' in selected_types:
        data.append(go.Histogram(x=current_total, name='Total', opacity=0.75))

    layout = go.Layout(
        title='Histogram of Different Types',
        xaxis=dict(title='Values'),
        yaxis=dict(title='How Many People'),
        barmode='overlay'
    )

    return {'data': data, 'layout': layout}

# Run the App
if __name__ == '__main__':
    app.run_server(debug=True)
